https://www.notion.so/Backtest-pnl-on-events-Low-skew-CB-Pivot-ATH-88eec344ef2743cebeba7136839e7169?pvs=4

In [10]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import py #import *
from py import *

In [11]:
# Example usage
# Generate a sample DataFrame with placeholder data
df = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'IV_OTM_Put': np.random.uniform(low=0.15, high=0.3, size=100),
    'IV_ATM': np.random.uniform(low=0.1, high=0.25, size=100),
    'IV_OTM_Call': np.random.uniform(low=0.15, high=0.3, size=100)
})

# Calculate the skew for all periods
df_with_skew = calculate_skew(df)

# Define your skew threshold
skew_threshold = 0.2

# Filter for periods where skew is below the threshold
low_skew_periods = filter_for_low_skew_periods(df_with_skew, skew_threshold)

low_skew_periods.head()

NameError: name 'calculate_skew' is not defined

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

# This function calculates the skewness levels based on percentile and standard deviation threshold
def calculate_skewness_levels(data, iv_put_col, iv_call_col, iv_atm_col, percentile=10, std_devs_below_mean=1):
    """
    Calculate the skewness levels based on the percentile and standard deviation threshold.
    
    :param data: DataFrame with columns for the implied volatilities for OTM puts, ATM options, and OTM calls.
    :param iv_put_col: Column name for implied volatilities of OTM puts.
    :param iv_call_col: Column name for implied volatilities of OTM calls.
    :param iv_atm_col: Column name for implied volatilities of ATM options.
    :param percentile: Percentile to determine the low skew level.
    :param std_devs_below_mean: Number of standard deviations below the mean to determine low skew level.
    :return: A dictionary with 'percentile_level' and 'std_dev_level' for skewness.
    """
    data['Skew'] = (data[iv_put_col] + data[iv_call_col]) / 2 - data[iv_atm_col]
    percentile_level = np.percentile(data['Skew'], percentile)
    mean_skew = data['Skew'].mean()
    std_dev_skew = data['Skew'].std()
    std_dev_level = mean_skew - std_devs_below_mean * std_dev_skew
    return {
        'percentile_level': percentile_level,
        'mean_skew': mean_skew,
        'std_dev_skew': std_dev_skew,
        'std_dev_level': std_dev_level
    }

# Generate a sample DataFrame with placeholder data
df = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=100, freq='D'),
    'IV_OTM_Put': np.random.normal(0.20, 0.05, 100),  # Placeholder data
    'IV_ATM': np.random.normal(0.18, 0.04, 100),      # Placeholder data
    'IV_OTM_Call': np.random.normal(0.22, 0.05, 100)  # Placeholder data
})

# Calculate the skewness levels
skewness_levels = calculate_skewness_levels(df, 'IV_OTM_Put', 'IV_OTM_Call', 'IV_ATM')

# Plot the skewness distribution and the calculated levels
plt.figure(figsize=(12, 6))
plt.hist(df['Skew'], bins=30, alpha=0.7, label='Skewness Distribution')
plt.axvline(skewness_levels['percentile_level'], color='red', linestyle='dashed', linewidth=2, label='10th Percentile')
plt.axvline(skewness_levels['mean_skew'], color='green', linestyle='dashed', linewidth=2, label='Mean Skew')
plt.axvline(skewness_levels['std_dev_level'], color='blue', linestyle='dashed', linewidth=2, label='1 Std Dev Below Mean')
plt.title('Implied Volatility Skewness Distribution')
plt.xlabel('Skewness Level')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Plot the skewness evolution over time with the mean and standard deviation lines
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Skew'], label='Daily Skew', color='black')
plt.axhline(skewness_levels['mean_skew'], color='green', linestyle='dashed', linewidth=2, label='Mean Skew')
plt.axhline(skewness_levels['std_dev_level'], color='blue', linestyle='dashed', linewidth=2, label='1 Std Dev Below Mean')
plt.fill_between(df['Date'], skewness_levels['mean_skew'] - skewness_levels['std_dev_skew'], 
                 skewness_levels['mean_skew'] + skewness_levels['std_dev_skew'], color='blue', alpha=0.1, label='1 Std Dev Range')
plt.title('Skewness Evolution Over Time')
plt.xlabel('Date')
plt.ylabel('Skewness Level')
plt.legend()
plt.show()


### Skew analysis
    Calculation of upskew, downskew, and overall skew.
    Determination of spread and ratio between downskew and upskew.
    Identification of periods where the spread or ratio exceeds certain thresholds.
    Calculation of average and median implied volatility shifts for various time intervals after threshold conditions are met.
    Visualization of skewness evolution over time.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

def calculate_skew(data, iv_put_col, iv_call_col, iv_atm_col):
    """
    Calculate the skew as the difference between the IV of OTM puts and calls relative to ATM options.
    """
    data['UpSkew'] = data[iv_call_col] - data[iv_atm_col]
    data['DownSkew'] = data[iv_atm_col] - data[iv_put_col]
    data['Skew'] = (data['IV_OTM_Put'] - data['IV_OTM_Call']) / data['IV_ATM']
    return data

def calculate_spreads_and_ratios(data):
    """
    Calculate the spread between downskew and upskew, and the ratio of upskew to skew.
    """
    data['Spread'] = data['DownSkew'] - data['UpSkew']
    data['Ratio'] = data['UpSkew'] / data['Skew'].replace(0, np.nan)  # Avoid division by zero
    return data

def find_threshold_periods(data, spread_threshold, ratio_threshold):
    """
    Find the periods where the spread and ratio meet certain thresholds.
    """
    condition = (data['Spread'] > spread_threshold) | (data['Ratio'] > ratio_threshold)
    return data[condition]

def calculate_vol_shifts(data, threshold_periods, shifts):
    """
    Calculate the average and median vol shifts after certain thresholds have been reached.
    """
    results = {}
    for shift in shifts:
        shift_days = shift * 30
        vol_shifts = []
        for date in threshold_periods['Date']:
            start_vol = data.loc[data['Date'] == date, 'IV_ATM'].values
            end_date = date + timedelta(days=shift_days)
            end_vol = data.loc[data['Date'] == end_date, 'IV_ATM'].values
            if start_vol.size > 0 and end_vol.size > 0:
                vol_shifts.append(end_vol[0] - start_vol[0])

        results[shift] = {'average': np.nanmean(vol_shifts), 'median': np.nanmedian(vol_shifts)}
    return pd.DataFrame(results)



In [ ]:
# Sample data generation and function calls
period = 150
df = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=period, freq='D'),
    'IV_OTM_Put': np.random.uniform(0.15, 0.3, period),
    'IV_ATM': np.random.uniform(0.1, 0.25, period),
    'IV_OTM_Call': np.random.uniform(0.15, 0.3, period)
})

df = calculate_skew(df, 'IV_OTM_Put', 'IV_OTM_Call', 'IV_ATM')
df = calculate_spreads_and_ratios(df)
threshold_periods = find_threshold_periods(df, 0.05, 1.5)
vol_shifts = calculate_vol_shifts(df, threshold_periods, [1, 3, 6, 9, 12])

# Plotting the evolution of skewness
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['UpSkew'], label='Upskew', color='blue')
plt.plot(df['Date'], df['DownSkew'], label='DownSkew', color='red')
plt.title('Skewness Evolution Over Time')
plt.xlabel('Date')
plt.ylabel('Skewness Level')
plt.legend()
plt.show()

print(vol_shifts)

In [ ]:
Identify Qualifying Periods